In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from scipy.spatial.distance import mahalanobis


# Week 5 Coding Quiz:

In [2]:
# Code to create data for this question 
num = 100000 
 
difficulty = np.random.uniform(0, 1, (num,)) 
 
speed = np.maximum(np.random.normal(15, 5, (num, )) - difficulty * 10, 0) 
 
accident = np.minimum(np.maximum(0.03 * speed + 0.4 * difficulty + np.random.normal(0, 0.3, (num,)), 0), 1) 
 
df = pd.DataFrame({'difficulty': difficulty, 'speed': speed, 'accident': accident}) 
df.head()


,difficulty,speed,accident
0,0.676866,6.669184,0.318685
1,0.634571,6.626761,0.496661
2,0.107245,21.476106,0.496126
3,0.144006,15.749975,0.196447
4,0.030893,14.013525,0.661416


In [3]:
# Question 1
def experiment(num=100000, seed=None):

    if seed is not None:
        np.random.seed(seed)

    # same as the provided code for the quiz
    difficulty = np.random.uniform(0, 1, (num,))
    speed = np.maximum(np.random.normal(15, 5, (num, )) - difficulty * 10, 0) 

    # regress speed based on the on difficulty
    X = sm.add_constant(difficulty)
    model = sm.OLS(speed, X).fit()
    # return the coefficient or whatever
    return model.params[1]

# run 500 experiments
coeffs = [experiment(seed=i) for i in range(500)]

# show the average, standard deviation of the coefficients
np.mean(coeffs), np.std(coeffs)

(np.float64(-9.663234560549936), np.float64(0.052062746858112914))

In [4]:
# Question 2

def simulate_once(num=100000):
    difficulty = np.random.uniform(0, 1, (num,))
    speed = np.maximum(np.random.normal(15, 5, (num, )) - difficulty * 10, 0) 
    accident = np.minimum(np.maximum(0.03 * speed + 0.4 * difficulty + np.random.normal(0, 0.3, (num,)), 0), 1) 
    df = pd.DataFrame({'difficulty': difficulty, 'speed': speed, 'accident': accident})
    
    # regress speed on difficulty and likelihood of acccident
    X = df[['difficulty', 'accident']]
    y = df['speed']
    model = LinearRegression().fit(X, y)
    return model.coef_[0]  # difficulty coefficient

# run 500 times to get the average slope
coeffs = [simulate_once() for _ in range(500)]

# show the average, standard deviation of the coefficients
np.mean(coeffs), np.std(coeffs)



(np.float64(-10.326134822154236), np.float64(0.044428368869339686))

# Week 6 Coding Quiz

In [5]:
# Load data and compute ATE, ATT, ATU, and "optimal treatment effect" per the prompt.

# Load the CSV
df = pd.read_csv("/Users/amit/Desktop/Current Classes/Y2S1 Experimental Design and Causality/DX702-mod-6/homework_6.1.csv")

# Expect columns: outcome Y, treatment X (0/1), confounder Z
# Try to guess column names robustly
cols = {c.lower(): c for c in df.columns}
Y_col = cols.get("y", None) or cols.get("outcome", None) or list(df.columns)[0]
X_col = cols.get("x", None) or cols.get("treatment", None) or list(df.columns)[1]
Z_col = cols.get("z", None) or cols.get("confounder", None) or list(df.columns)[2]

df = df[[Y_col, X_col, Z_col]].rename(columns={Y_col:"Y", X_col:"X", Z_col:"Z"}).copy()

# Split groups
treated = df[df["X"] == 1].reset_index(drop=True)
control = df[df["X"] == 0].reset_index(drop=True)

# Helper: fit 1-NN on Z arrays
def nearest_index(src_Z, dst_Z):
    # src_Z: array of shape (n_src, 1); dst_Z: (n_dst, 1)
    nn = NearestNeighbors(n_neighbors=1, metric="euclidean")
    nn.fit(dst_Z)
    distances, indices = nn.kneighbors(src_Z, return_distance=True)
    return indices.ravel()

# For each treated unit, find nearest control by Z
t_to_c_idx = nearest_index(treated[["Z"]].values, control[["Z"]].values)
treated_matches = control.iloc[t_to_c_idx].reset_index(drop=True)

# For each control unit, find nearest treated by Z
c_to_t_idx = nearest_index(control[["Z"]].values, treated[["Z"]].values)
control_matches = treated.iloc[c_to_t_idx].reset_index(drop=True)

# Individual effects:
# For treated i: tau_i = Y_treated_i - Y_matched_control_i
tau_treated = treated["Y"].values - treated_matches["Y"].values

# For control j: tau_j = Y_matched_treated_j - Y_control_j
tau_control = control_matches["Y"].values - control["Y"].values

# ATE: average over *all items* using their nearest opposite-group counterfactual
ATE = np.mean(np.concatenate([tau_treated, tau_control]))

# ATT: average over treated only
ATT = np.mean(tau_treated)

# ATU: average over untreated only
ATU = np.mean(tau_control)

# "Optimal treatment effect": maximum treatment effect across all untreated items
# i.e., consider only untreated items' tau and take max
OPT = np.max(tau_control)

# Round for display
def r(x): 
    return float(np.round(x, 6))

results = {
    "n_treated": int(len(treated)),
    "n_control": int(len(control)),
    "ATE": r(ATE),
    "ATT": r(ATT),
    "ATU": r(ATU),
    "OPT_effect_over_untreated_max": r(OPT),
}

results


{'n_treated': 491,
 'n_control': 509,
 'ATE': 1.69527,
 'ATT': 1.846409,
 'ATU': 1.549477,
 'OPT_effect_over_untreated_max': 2.17247}

# Week 7 Coding Quiz

In [6]:
# Question 1

# question data generation
np.random.seed(0)
W = np.random.normal(0, 1, (1000,))
X = W + np.random.normal(0, 1, (1000,))
Z = np.random.normal(0, 1, (1000,)) 
Y = X + Z + W + np.random.normal(0, 1, (1000,))

# fit the linear model (Y ~ X + Z + W) and compute residuals
X_mat = np.column_stack([np.ones(len(X)), X, Z, W])  # add intercept
beta_hat = np.linalg.inv(X_mat.T @ X_mat) @ X_mat.T @ Y
Y_hat = X_mat @ beta_hat
residuals = Y - Y_hat

# compute correlation between X and residuals (error term)
corr = np.corrcoef(X, residuals)[0, 1]
print(f"Correlation between X and residuals: {corr:.4f}")


Correlation between X and residuals: 0.0000


In [7]:
# Question 2

# question data
W = np.random.normal(0, 1, (1000,))
X = W + np.random.normal(0, 1, (1000,))
Z = np.random.normal(0, 1, (1000,))
Y = X + Z + W + np.random.normal(0, 1, (1000,))

eps = Y - (X + Z + W)
u_true = W + eps
corr_struct = np.corrcoef(X, u_true)[0, 1]
print(f"Corr(X, true error u=W+eps): {corr_struct:.4f}")

Xmat = np.column_stack([np.ones_like(X), X, Z])
beta = np.linalg.inv(Xmat.T @ Xmat) @ (Xmat.T @ Y)
Yhat = Xmat @ beta
resid = Y - Yhat

corr_resid = np.corrcoef(X, resid)[0, 1]
print(f"Corr(X, OLS residuals from Y~X+Z): {corr_resid:.4f}")


Corr(X, true error u=W+eps): 0.5090
Corr(X, OLS residuals from Y~X+Z): -0.0000


In [8]:
df = pd.read_csv("/Users/amit/Desktop/Current Classes/Y2S1 Experimental Design and Causality/DX702-mod-6/homework_7.1.csv")
df

,Unnamed: 0,X,W,Z,Y
0,0,1.137055,1.221768,0.327829,1.944532
1,1,-0.112905,0.465835,0.599650,0.655514
2,2,2.077755,1.795414,-0.063393,5.934411
3,3,0.456373,-0.512159,1.177413,-0.188064
4,4,-1.012402,0.080002,-0.275697,-0.533775
...,...,...,...,...,...
9995,9995,2.569934,1.233620,0.930467,6.188783
9996,9996,0.190150,1.022164,-0.015151,0.697187
9997,9997,-1.184465,-1.475929,-0.287056,-1.575303
9998,9998,-0.121286,-0.914357,1.706237,-1.809819


In [9]:
# Question 3

# Compute beta_X at W≈-1,0,1 from homework_7.1.csv and summarize the trend

# Load
df = pd.read_csv("/Users/amit/Desktop/Current Classes/Y2S1 Experimental Design and Causality/DX702-mod-6/homework_7.1.csv")[["X", "Y","Z","W"]].dropna()

def ols_beta_x(sub):
    Xmat = np.column_stack([np.ones(len(sub)), sub["X"].values, sub["Z"].values])
    yvec = sub["Y"].values
    b, *_ = np.linalg.lstsq(Xmat, yvec, rcond=None)
    return b[1]

targets = [-1.0, 0.0, 1.0]
h = 0.25

rows = []
for w0 in targets:
    mask = (df["W"] >= w0 - h) & (df["W"] <= w0 + h)
    sub = df.loc[mask]
    beta_x = ols_beta_x(sub) if len(sub) >= 5 else np.nan
    rows.append({"W≈": w0, "n": len(sub), "beta_X_local(h=0.25)": beta_x})

# Global interaction check
X_full = np.column_stack([
    np.ones(len(df)),
    df["X"].values,
    df["Z"].values,
    df["W"].values,
    (df["X"]*df["W"]).values,
    (df["Z"]*df["W"]).values,
])
y_full = df["Y"].values
b, *_ = np.linalg.lstsq(X_full, y_full, rcond=None)
b0,bX,bZ,bW,bXW,bZW = b

for w0 in targets:
    rows.append({"W≈": w0, "n": len(df), "beta_X_local(h=0.25)": np.nan,
                 "beta_X_from_interaction": bX + bXW*w0})

out = pd.DataFrame(rows)

# Pivot for readability
pivot = out.pivot_table(index="W≈", values=["beta_X_local(h=0.25)","beta_X_from_interaction"], aggfunc="mean")
pivot_rounded = pivot.round(3)
pivot_rounded


,beta_X_from_interaction,beta_X_local(h=0.25)
W≈,,
-1.0,0.900,0.902
0.0,1.396,1.418
1.0,1.893,1.920


In [10]:
# Question 4
# Simulation to study effect of serial correlation strength on beta_X variability vs. its OLS SE
import numpy as np
import pandas as pd

rng = np.random.default_rng(42)

def make_error(corr_const, num, rng):
    """AR(1)-style error with parameter corr_const and unit-variance innovations."""
    out = np.empty(num, dtype=float)
    prev = rng.normal(0, 1)
    for i in range(num):
        prev = corr_const * prev + (1 - corr_const) * rng.normal(0, 1)
        out[i] = prev
    return out

def ols_beta_and_se(y, x):
    """Return beta1 and its conventional SE from OLS of y ~ 1 + x."""
    n = len(x)
    X = np.column_stack([np.ones(n), x])
    # OLS via normal equations
    XtX = X.T @ X
    XtX_inv = np.linalg.inv(XtX)
    beta = XtX_inv @ (X.T @ y)
    resid = y - X @ beta
    dof = n - X.shape[1]
    sigma2 = (resid @ resid) / dof
    se = np.sqrt(sigma2 * XtX_inv[1, 1])  # SE for beta_x
    return beta[1], se

def run_trials(rho, n=500, trials=800):
    betas = np.empty(trials)
    ses = np.empty(trials)
    for t in range(trials):
        eps_x = make_error(rho, n, rng)
        eps_y = make_error(rho, n, rng)  # independent of eps_x innovations
        X = 0.5 + eps_x
        beta_true = 1.0
        Y = 2.0 + beta_true * X + eps_y
        b1, se1 = ols_beta_and_se(Y, X)
        betas[t] = b1
        ses[t] = se1
    sd_b = betas.std(ddof=1) # 1. empirical SD of beta_hat
    mean_se = ses.mean() # 2. mean of estimated SE
    ratio = sd_b / mean_se
    return sd_b, mean_se, ratio

results = []
for rho in [0.2, 0.5, 0.8]:
    sd_b, mean_se, ratio = run_trials(rho, n=500, trials=800)
    results.append({"corr_const": rho, "sd_beta_hat (i)": sd_b, "mean_SE_hat (ii)": mean_se, "ratio (i)/(ii)": ratio})

df_res = pd.DataFrame(results).round(4)
df_res


,corr_const,sd_beta_hat (i),mean_SE_hat (ii),ratio (i)/(ii)
0,0.2,0.0480,0.0447,1.0735
1,0.5,0.0576,0.0449,1.2823
2,0.8,0.0993,0.0448,2.2195


# Week 8 Coding Quiz

In [11]:
# Q1 

path = "/Users/amit/Desktop/Current Classes/Y2S1 Experimental Design and Causality/DX702-mod-6/homework_8.1.csv"
df = pd.read_csv(path)

df2 = df.drop(columns=[c for c in df.columns if c.lower().startswith("unnamed")], errors="ignore").copy()

# predict X from Zfor propensity model 
X_feat = df2[["Z"]].values
treat = df2["X"].values

# logistic regression
logit = LogisticRegression(solver="lbfgs")
logit.fit(X_feat, treat)

# predict propensity scores
propensity = logit.predict_proba(X_feat)[:, 1]

# avoid division by zero in weights
eps = 1e-6
p = np.clip(propensity, eps, 1 - eps)

# IPW estimates
Y = df2["Y"].values
Ey1 = np.mean((treat * Y) / p)            
Ey0 = np.mean(((1 - treat) * Y) / (1 - p))
ate_ipw = Ey1 - Ey0




diff_means = df2.loc[df2["X"]==1, "Y"].mean() - df2.loc[df2["X"]==0, "Y"].mean()

pi = treat.mean()
sw_treated = pi / p
sw_control = (1 - pi) / (1 - p)
Ey1_sw = np.sum(sw_treated * treat * Y) / np.sum(sw_treated * treat + 1e-12)
Ey0_sw = np.sum(sw_control * (1 - treat) * Y) / np.sum(sw_control * (1 - treat) + 1e-12)
ate_sw = Ey1_sw - Ey0_sw





print("Propensity summary: min={:.4f}, mean={:.4f}, max={:.4f}".format(p.min(), p.mean(), p.max()))
print("E[Y^1] (IPW) =", Ey1)
print("E[Y^0] (IPW) =", Ey0)
print("ATE (IPW)    =", ate_ipw)

print("Unweighted diff-in-means =", diff_means)
print("Stabilized IPW ATE       =", ate_sw)


Propensity summary: min=0.0480, mean=0.4810, max=0.9322
E[Y^1] (IPW) = 2.2010396833844457
E[Y^0] (IPW) = -0.03821919617103208
ATE (IPW)    = 2.2392588795554778
Unweighted diff-in-means = 3.036626208813963
Stabilized IPW ATE       = 2.274341189846216


In [12]:
# Q2 
print(f"First 3 propensities: {propensity[:3]}")

First 3 propensities: [0.84011371 0.58464597 0.71108245]


In [13]:
# Q3

path = "/Users/amit/Desktop/Current Classes/Y2S1 Experimental Design and Causality/DX702-mod-6/homework_8.2.csv"
df = pd.read_csv(path)

# clean
df2 = df.drop(columns=[c for c in df.columns if c.lower().startswith("unnamed")], errors="ignore").copy()


# Build inverse covariance from Z1, Z2
if not {"Z1", "Z2", "X", "Y"}.issubset(df.columns):
    raise ValueError(f"Expected columns X, Y, Z1, Z2. Found: {list(df.columns)}")
Z = df[["Z1", "Z2"]].copy()
Z_rows = np.vstack([Z["Z1"].values, Z["Z2"].values])  
cov_2x2 = np.cov(Z_rows)
VI = np.linalg.inv(cov_2x2)



# Match treated to nearest control (mahalanobis, with replacement)
treated_idx = df.index[df["X"] == 1].to_numpy()
control_idx = df.index[df["X"] == 0].to_numpy()

Z_t = Z.loc[treated_idx].to_numpy()
Z_c = Z.loc[control_idx].to_numpy()   # shape: n_c x 2

Y_t = df.loc[treated_idx, "Y"].to_numpy()
Y_c_all = df.loc[control_idx, "Y"].to_numpy()

# For each treated point, find single nearest control by Mahalanobis distance
match_control_indices = []
for i in range(Z_t.shape[0]):
    zi = Z_t[i]
    # compute distances to all controls
    dists = np.array([mahalanobis(zi, Z_c[j], VI) for j in range(Z_c.shape[0])])
    j_best = np.argmin(dists)
    match_control_indices.append(control_idx[j_best])

matched_controls_y = df.loc[match_control_indices, "Y"].to_numpy()

ate_match = float(np.mean(Y_t - matched_controls_y))
print("Mahalanobis matching ATE (treated → nearest control, with replacement):", ate_match)




Mahalanobis matching ATE (treated → nearest control, with replacement): 3.437678997912609


In [14]:
# Q4
df = df.drop(columns=[c for c in df.columns if c.lower().startswith("unnamed")], errors="ignore")

# Expect columns: X (treatment), Y (outcome), Z1, Z2 (covariates)
assert {"X","Z1","Z2"}.issubset(df.columns), f"Expected columns X,Z1,Z2; found {df.columns.tolist()}"

Z = df[["Z1", "Z2"]].copy()
Z_rows = np.vstack([Z["Z1"].values, Z["Z2"].values])
cov_2x2 = np.cov(Z_rows)
VI = np.linalg.inv(cov_2x2)

treated = df[df["X"]==1].copy()
control = df[df["X"]==0].copy()

Z_t = treated[["Z1","Z2"]].to_numpy()
Z_c = control[["Z1","Z2"]].to_numpy()

nearest_dist = []
nearest_control_idx = []
for i in range(Z_t.shape[0]):
    zi = Z_t[i]
    dists = np.array([mahalanobis(zi, Z_c[j], VI) for j in range(Z_c.shape[0])])
    j_best = int(np.argmin(dists))
    nearest_control_idx.append(j_best)
    nearest_dist.append(float(dists[j_best]))

nearest_dist = np.array(nearest_dist)
worst_idx_local = int(np.argmax(nearest_dist))   # index within treated subset
worst_treated_row = treated.iloc[worst_idx_local]

worst_point = (float(worst_treated_row["Z1"]), float(worst_treated_row["Z2"]))
worst_distance = float(nearest_dist[worst_idx_local])

print("Worst treated point (Z1, Z2):", worst_point)
print("Max nearest-control Mahalanobis distance:", worst_distance)

Worst treated point (Z1, Z2): (2.6962240525635797, 0.5381554886023228)
Max nearest-control Mahalanobis distance: 1.3830045328325054
